##MCP SERVER

In [ ]:
!pip install fastapi uvicorn pyngrok unsloth

-setup for the server in colllab becuse i dont have a gpu in my computer to run the model

---

i have created a simple server to show how it works, after starting the app
a tunnel is made using ngrok to thier ip adress the app is in the url/docs. i have left my token so you wont need to create an account



In [7]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class PromptRequest(BaseModel):
    context: str

@app.post("/tool/get_elaborate_description_prompt")
def get_prompt(req: PromptRequest):
    return {"prompt": f"Elaborated: yes itsa mi mario"}

In [8]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.kill()
# Allow running asyncio inside Jupyter
ngrok.set_auth_token("2zHIdB46Eante8AXvCi2J0ZquJq_5SCxEYeeMJqr56Ahz6Jz1")
nest_asyncio.apply()
# Open tunnel to port 8000
public_url = ngrok.connect(8000)
print(f"🔗 Public URL: {public_url}")

🔗 Public URL: NgrokTunnel: "https://0698-35-233-254-162.ngrok-free.app" -> "http://localhost:8000"


cache.py

In [ ]:
import csv
from unsloth import FastLanguageModel
from peft import PeftModel
import torch
from google.colab import files
import os
import zipfile

# Simulate cache for concept descriptions
description_cache = {}
model_name = "finetuned_phi4"
model_type = "unsloth/Phi-4"

def generate_elaborate_prompt(context: str) -> str:
  print("starting model")
  prompt = f"### Instruction:\n{context.strip()}\n### Response:\n"
  print("prompt")
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  with torch.no_grad():  # Avoids unnecessary gradient tracking
    outputs = model.generate(**inputs, max_new_tokens=100)
  full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return full_output.split("### Response:")[-1].strip()

def cache_description(concept: str, prompt: str):
  description_cache[concept] = prompt

def get_cached_description(concept: str) -> str:
  return description_cache.get(concept)

def get_all_cached_descriptions() -> dict:
  return description_cache

def load_cache_from_csv(file_path: str):
  with open(file_path, newline='', encoding='utf-8') as csvfile:
      reader = csv.DictReader(csvfile)
      for row in reader:
          concept = row["input"]
          prompt = row["finetuned_model_answer"]
          if concept and prompt:
              cache_description(concept, prompt)



def load_model():
  global model, tokenizer
  model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_type,
    max_seq_length=2048,
    load_in_4bit=True,         
    device_map="auto",         
    dtype=None,
    )
  model = PeftModel.from_pretrained(model, model_name)
  model.eval()
  tokenizer = tokenizer.from_pretrained(model_name)


models.py

In [13]:
from pydantic import BaseModel

class ContextRequest(BaseModel):
    context: str

class DescriptionResponse(BaseModel):
    prompt: str

class CacheRequest(BaseModel):
    id: str

class CacheListResponse(BaseModel):
    descriptions: dict

class ContextRequest(BaseModel):
    context: str

main.py

In [14]:
from fastapi import FastAPI, HTTPException


app = FastAPI()


@app.on_event("startup")
def startup_event():
    load_cache_from_csv("similarity_results_train.csv")
    load_model()


@app.post("/tool/get_elaborate_description_prompt", response_model=DescriptionResponse)
def get_elaborate_description_prompt(request: ContextRequest):
    prompt = generate_elaborate_prompt(request.context)
    return {"prompt": prompt}

@app.get("/resource/cached_description/{concept_id}", response_model=DescriptionResponse)
def cached_description(concept_id: str):
    prompt = get_cached_description(concept_id)
    if prompt is None:
        raise HTTPException(status_code=404, detail="Concept ID not found in cache")
    return {"prompt": prompt}

@app.get("/resource/cached_description_list", response_model=CacheListResponse)
def cached_description_list():
    return {"descriptions": get_all_cached_descriptions()}

/tmp/ipython-input-14-4186044854.py:7: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.kill()
nest_asyncio.apply()
ngrok.set_auth_token("2zHIdB46Eante8AXvCi2J0ZquJq_5SCxEYeeMJqr56Ahz6Jz1")

# Expose port 8000
public_url = ngrok.connect(8000)
print(f"🔗 Server ready at: {public_url}/docs")

# Start your FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)